### Import libraries

In [115]:
import pysftp
import pyodbc as odbc
import re
import numpy as np
import pandas as pd
from itertools import combinations
from datetime import datetime, timedelta
from IPython.display import Image
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings
import warnings
disable_warnings(InsecureRequestWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
pd.options.mode.chained_assignment = None

### Functions used in a project

In [392]:
#traversing iterator through list of lists
def traverse_objects(obj, types = (list)):
    if isinstance(obj, types):
        for value in obj:
            for subvalue in traverse_objects(value, types):
                yield subvalue
    else:
        yield obj
        
#generating dataframe from data collected in a dictionary for adding them to sql database        
def dict_to_df(dictionary, df_column_list):
    data_frame = pd.DataFrame(columns = df_column_list)

    for item in dictionary.items():
        for value in item[1]:
            data_frame.loc[-1] = 0
            data_frame[df_column_list[0]].loc[-1] = item[0]
            data_frame[df_column_list[1]].loc[-1] = value
            data_frame.index = data_frame.index + 1
    data_frame = data_frame.where(pd.notnull(data_frame), None)
    data_frame = data_frame.sort_values(by=df_column_list).reset_index()
    data_frame = data_frame.drop(['index'], axis=1)
    return data_frame

### Import hdf from sftp server

In [393]:
sftpHost = "localhost"
sftpPort = 22
uname = "wojci"
privateKeyFilePath = "./id_rsa"

cnOpts = pysftp.CnOpts()
cnOpts.hostkeys = None

with pysftp.Connection(host=sftpHost, username=uname, private_key=privateKeyFilePath, cnopts=cnOpts) as sftp:
    print("Connected to sftp server")
    sftp.cwd("./beer_files")
    sftp.get("./beer_files.h5", "./downloaded_from_sftp/beers.h5", preserve_mtime=True)
    print('File "beers.h5" downloaded correctly')

Connected to sftp server
File "beers.h5" downloaded correctly


### Read imported file as DataFrame

In [394]:
df = pd.read_hdf('./downloaded_from_sftp/beers.h5', key='df', mode='r')
pd.set_option('display.max_columns', 27) #show all columns
df = df.where(pd.notnull(df), None)
df

,beer_name,name of style,name_of_substyle,beer_country,beer_state,rank_in_style,score,rank,abv [%],avg_score,deviation_score [%],ratings,active,date_added,who_wants,who_have,brewery_name,brewery_city,brewery_state,brewery_country,brewery_map,brewery_website,brewery_type,brewery_adress,brewery_postal_code,brewery_telephone,brewery_notes
0,Ayinger Celebrator,Bocks,Bock - Doppelbock,Germany,None,1.0,96.0,879.0,6.7,4.34,9.91,6978,1,2001-01-10,623,1142,Ayinger Privatbrauerei,Aying,None,Germany,https://maps.google.com/maps?oi=map&q=M%C3%BCn...,http://ayinger-bier.de,"[Brewery, Bar, Eatery]",Münchener Straße 21,85653,08095-90650,None
1,Troegenator,Bocks,Bock - Doppelbock,United States,Pennsylvania,48.0,88.0,13560.0,8.2,3.95,11.65,3516,1,2003-02-12,163,845,Tröegs Brewing Company,Hershey,Pennsylvania,United States,https://maps.google.com/maps?oi=map&q=200+East...,http://troegs.com,"[Brewery, Bar, Eatery, Beer-to-go]",200 East,17033,(717) 534-1297,Sunday-Wednesday 11am-9pmThursday - sat 11am-1...
2,Spaten Optimator,Bocks,Bock - Doppelbock,Germany,None,75.0,87.0,16263.0,7.6,3.90,12.05,3264,1,1999-04-06,114,518,Spaten-Franziskaner-Bräu,München,None,Germany,https://maps.google.com/maps?oi=map&q=Mars+Str...,http://franziskaner-weissbier.de,[Brewery],Mars Strasse 46-48,80335,(089) 51 221,OWNED BY ANHEUSER-BUSCH INBEVSee also: http://...
3,Salvator,Bocks,Bock - Doppelbock,Germany,None,57.0,88.0,14654.0,7.9,3.93,11.70,3047,1,2001-04-23,111,511,Paulaner Brauerei,München,None,Germany,https://maps.google.com/maps?oi=map&q=Hochstra...,http://paulaner.de,[Brewery],Hochstraße 75,81541,089 / 4 80 051,None
4,Weihenstephaner Korbinian,Bocks,Bock - Doppelbock,Germany,None,5.0,93.0,2924.0,7.4,4.20,9.05,2969,1,2001-09-12,251,398,Bayerische Staatsbrauerei Weihenstephan,Freising,None,Germany,https://maps.google.com/maps?oi=map&q=Alte+Aka...,http://weihenstephaner.de,"[Brewery, Bar, Eatery, Beer-to-go]",Alte Akademie 2,85354,+49 8161 5360,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180688,Dos Bayas Sour Ale (Strawberry/Key Lime),Wild/Sour Beers,Wild Ale,United States,Maryland,NaN,NaN,NaN,4.7,0.00,0.00,0,1,2020-05-12,0,0,Alecraft Brewery,Bel Air,Maryland,United States,https://maps.google.com/maps?oi=map&q=319+S+Ma...,http://alecraftbrewery.com,"[Brewery, Bar, Beer-to-go]",319 S Main St,21014,(410) 420-5102,Hours:Monday: Closed Tuesday: 11AM - 10PM Wedn...
180689,Bericluster,Wild/Sour Beers,Wild Ale,United States,Washington,NaN,NaN,NaN,5.8,0.00,0.00,0,1,2020-07-14,0,0,Tin Dog Brewing,Seattle,Washington,United States,https://maps.google.com/maps?oi=map&q=309+S+Cl...,http://tindogbrewing.com,"[Brewery, Bar, Beer-to-go]",309 S Cloverdale St. Unit A2,98108,None,None
180690,5-Point-5,Wild/Sour Beers,Wild Ale,United States,Wisconsin,NaN,NaN,NaN,5.5,0.00,0.00,0,1,2020-08-13,0,0,Broken Bat Brewery,Milwaukee,Wisconsin,United States,https://maps.google.com/maps?oi=map&q=135+E+Pi...,http://brokenbatbrewery.com,"[Brewery, Bar, Beer-to-go]",135 E Pittsburgh AveLower Level,53204,(414) 316-9197,None
180691,Motorboat Red,Wild/Sour Beers,Wild Ale,United States,Florida,NaN,NaN,NaN,5.2,0.00,0.00,0,1,2020-08-25,0,0,Bone Hook Brewing Co.,Naples,Florida,United States,https://maps.google.com/maps?oi=map&q=1514+Imm...,None,"[Brewery, Bar, Beer-to-go]","1514 Immokalee Rd, Ste 106",34110,(239) 631-8522,None


### Lists with unique values for certain columns which will be used as a dimensions in sql database

In [395]:
#dict of countries and states
country_states_dict = dict.fromkeys(list(df['beer_country'].unique()), None)
for key in country_states_dict:
    mask_country = df['beer_country'] == key
    country_states_dict[key] = list(df['beer_state'].loc[mask_country].unique())

#dict of styles and substyles
beer_styles_list = list(df['name of style'].unique())
beer_styles_dict = dict.fromkeys(beer_styles_list, None)
for key in beer_styles_dict:
    mask_style = df['name of style'] == key
    beer_styles_dict[key] = list(df['name_of_substyle'].loc[mask_style].unique())
    
#list of abv
abv_list = list(df['abv [%]'].unique())

#list of unique brewery types
brewery_types_unique_list = []
for value in traverse_objects(list(df['brewery_type'])):
    if value.strip() not in brewery_types_unique_list:
        brewery_types_unique_list.append(value.strip())

### Connect to SQL server
- beer_db and user was created by Microsoft SQL Server Managment Studio

In [8]:
DRIVER_NAME = 'ODBC Driver 17 for SQL Server'
SERVER_NAME = 'LAPTOP-N14CM24V'
DATABASE_NAME = 'beer_db'
USERNAME = 'wojtek'
PASSWORD = 'password'

connection_string = f"""
DRIVER={{{DRIVER_NAME}}};
SERVER={SERVER_NAME};
DATABASE={DATABASE_NAME};
Trusted_Connection=yes;
UID={USERNAME};
PWD={PASSWORD};
"""

conn = odbc.connect(connection_string)
print(conn)
cursor = conn.cursor()

### Create tables in the beer_db

In [374]:
# Creating dimensional model, according to the structure of the obtained data. It will be easy snowflake schema.
# We can still do the same schema via SQL Server Managment Studio. I used SSMS for create a query,
# because jupyter notebook does not support highlighting errors in t-sql syntax

create_dimension_table_query = """
IF Object_ID('style_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[style_DIM]
(
    [PK_style_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [style_name] VARCHAR(100) NOT NULL,
    [substyle_name] VARCHAR(100) NOT NULL,
    )
    
IF Object_ID('country_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[country_DIM]
(
    [PK_country_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [country_name] VARCHAR(100) NOT NULL,
    [state_name] VARCHAR(100) NULL,
    )
     
IF Object_ID('abv_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[abv_DIM]
(
    [PK_abv_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [abv_value] FLOAT UNIQUE NULL,
    )
     
IF Object_ID('brewery_type_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[brewery_type_DIM]
(
    [PK_brewery_type_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [is_brewery] BIT NOT NULL,
    [is_bar] BIT NOT NULL,
    [is_eatery] BIT NOT NULL,
    [is_beer_to_go] BIT NOT NULL,
    [is_homebrew] BIT NOT NULL,
    )     
     
IF Object_ID('date_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[date_DIM]
(
    [PK_date_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [date] DATE UNIQUE NULL,
    )   
     
IF Object_ID('city_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[city_DIM]
(
    [PK_city_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [city] NVARCHAR(50) UNIQUE NULL,
    )   
     
IF Object_ID('brewery_DIM') IS NULL
CREATE TABLE [beer_db].[dbo].[brewery_DIM]
(
    [PK_brewery_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [FK_country_id] INT FOREIGN KEY REFERENCES country_DIM(PK_country_id),
    [FK_city_id] INT FOREIGN KEY REFERENCES city_DIM(PK_city_id),
    [FK_brewery_type_id] INT FOREIGN KEY REFERENCES brewery_type_DIM(PK_brewery_type_id),
    [brewery_name] NVARCHAR(100) NULL,
    [website] VARCHAR(50) UNIQUE NULL,
    [adress] NVARCHAR(255) UNIQUE NULL,
    [postal_code] VARCHAR(30) NULL,
    [telephone] VARCHAR(30) UNIQUE NULL,
    [map] VARCHAR(255) NULL UNIQUE,
    [brewery_notes] VARCHAR(max) NULL,
    )
     
IF Object_ID('beer') IS NULL
CREATE TABLE [beer_db].[dbo].[beer]
(
    [PK_beer_id] INT NOT NULL IDENTITY(1,1) PRIMARY KEY,
    [FK_brewery_id] INT FOREIGN KEY REFERENCES brewery_DIM(PK_brewery_id),
    [FK_style_id] INT FOREIGN KEY REFERENCES style_DIM(PK_style_id),
    [FK_abv_id] INT FOREIGN KEY REFERENCES abv_DIM(PK_abv_id),
    [FK_date_id] INT FOREIGN KEY REFERENCES date_DIM(PK_date_id),
    [beer_name] NVARCHAR(100) NULL,
    [rank_in_style] INT NULL,
    [score] INT NULL,
    [rank] INT NULL,
    [avg_score] FLOAT NULL,
    [dev_score] FLOAT NULL,
    [rating] INT NOT NULL,
    [who_want] INT NOT NULL,
    [who_have] INT NOT NULL,
    [active] BIT NOT NULL,
    )
"""

# Create the Table.
cursor.execute(create_dimension_table_query)
cursor.commit()

### Delete generated tables - only for functional tests purposes

In [373]:
delete_table_query = """
IF Object_ID('beer') IS NOT NULL
DROP TABLE [beer_db].[dbo].[beer]

IF Object_ID('brewery_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[brewery_DIM]

IF Object_ID('style_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[style_DIM]

IF Object_ID('country_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[country_DIM]

IF Object_ID('abv_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[abv_DIM]

IF Object_ID('brewery_type_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[brewery_type_DIM]

IF Object_ID('date_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[date_DIM]

IF Object_ID('city_DIM') IS NOT NULL
DROP TABLE [beer_db].[dbo].[city_DIM]
"""
    
# Delete tables.
cursor.execute(delete_table_query)
cursor.commit()

### Dimensional schema generated for the database

![title](documentation/img/dimensional_schema.jpg)

### Feed dimension tables without brewery_DIM

#### Prepare data

In [396]:
#creating data for feeding abv_DIM table
abv_DIM_array = np.arange(0, max(abv_list) + 8.1, 0.1)
abv_DIM_array = np.around(abv_DIM_array, 1)
abv_DIM_list = [(x,) for x in abv_DIM_array]

#looking for all possible combinations of brewery types
unique_brewery_types_combinations = []

for i in range(len(brewery_types_unique_list)+1):
    unique_brewery_types_combinations.append(list(combinations(brewery_types_unique_list, i)))

unique_brewery_types_combinations_list = list(traverse_objects(unique_brewery_types_combinations))

#creating dataframe for feeding brewery_type_DIM table
brewery_type_DIM_df = pd.DataFrame(columns = unique_brewery_types_combinations_list[-1])

for i in unique_brewery_types_combinations_list:
    brewery_type_DIM_df.loc[-1] = 0
    for x in brewery_types_unique_list:
        if x in i:
            brewery_type_DIM_df[str(x)].loc[-1] = 1
    brewery_type_DIM_df.index = brewery_type_DIM_df.index + 1
brewery_type_DIM_df = brewery_type_DIM_df.sort_index()
brewery_type_DIM_list = brewery_type_DIM_df.values.tolist()

#creating data for feeding city_DIM
city_DIM = sorted(df['brewery_city'].unique())
city_DIM_list = [[x] for x in city_DIM]

#creating dataframe for feeding country_DIM table
country_DIM_df = dict_to_df(country_states_dict, ['country_name', 'state_name'])
country_DIM_list = country_DIM_df.values.tolist()

#creating data for feeding date_DIM
date_DIM = pd.date_range(start=sorted(df['date_added'].unique())[0], end='2050-12-31')
date_DIM_list = [[x] for x in date_DIM]

#creating dataframe for feeding style_DIM 
style_DIM_df = dict_to_df(beer_styles_dict, ['style_name', 'substyle_name'])
style_DIM_list = style_DIM_df.values.tolist()

#### Make query

In [375]:
#abv_DIM table insert query
abv_DIM_insert = """
INSERT INTO [beer_db].[dbo].[abv_DIM]
(
    [abv_value]
)
VALUES
(
    ?
)
"""

#brewery_type_DIM table insert query
brewery_type_DIM_insert = """
INSERT INTO [beer_db].[dbo].[brewery_type_DIM]
(
    [is_brewery],
    [is_bar],
    [is_eatery],
    [is_beer_to_go],
    [is_homebrew]
)
VALUES
(
    ?,?,?,?,?
)
"""

#city_DIM table insert query
city_DIM_insert = """
INSERT INTO [beer_db].[dbo].[city_DIM]
(
    [city]
)
VALUES
(
    ?
)
"""

#country_DIM table insert query
country_DIM_insert = """
INSERT INTO [beer_db].[dbo].[country_DIM]
(
    [country_name],
    [state_name]
)
VALUES
(
    ?,?
)
"""

#date_DIM table insert query
date_DIM_insert = """
INSERT INTO [beer_db].[dbo].[date_DIM]
(
    [date]
)
VALUES
(
    ?
)
"""

#date_DIM table insert query
style_DIM_insert = """
INSERT INTO [beer_db].[dbo].[style_DIM]
(
    [style_name],
    [substyle_name]
)
VALUES
(
    ?,?
)
"""

cursor.executemany(abv_DIM_insert, abv_DIM_list)
cursor.executemany(brewery_type_DIM_insert, brewery_type_DIM_list)
cursor.executemany(city_DIM_insert, city_DIM_list)
cursor.executemany(country_DIM_insert, country_DIM_list)
cursor.executemany(date_DIM_insert, date_DIM_list)
cursor.executemany(style_DIM_insert, style_DIM_list)

cursor.commit()



### Feed brewery_DIM

#### Cut data from main dataframe to brewery dataframe - each row refers to the different brewery

In [447]:
#list of unique breweries
brewery_list = list(df['brewery_name'].unique()) 

#creating brewery dataframe
brewery_df = pd.DataFrame(columns = ['brewery_country',
                                     'brewery_state',
                                     'brewery_city',
                                     'brewery_type',
                                     'brewery_name',
                                     'brewery_website',
                                     'brewery_adress',
                                     'brewery_postal_code',
                                     'brewery_telephone',
                                     'brewery_map',
                                     'brewery_notes'])

for brewery in brewery_list:
    brewery_df.loc[-1] = df.loc[df['brewery_name'] == brewery].iloc[0]
    brewery_df.index = brewery_df.index + 1

brewery_df = brewery_df.reset_index()
brewery_df = brewery_df.drop(['index'], axis=1)

#### Export brewery_df into hdf fine - only for time saving during work with the project

In [450]:
brewery_df.to_hdf('./hdf_files/brewery_df.h5', key='df', mode='w')
brewery_df = pd.read_hdf('./hdf_files/brewery_df.h5', key='df', mode='r')
brewery_df.head()

,brewery_country,brewery_state,brewery_city,brewery_type,brewery_name,brewery_website,brewery_adress,brewery_postal_code,brewery_telephone,brewery_map,brewery_notes
0,Germany,None,Aying,"[Brewery, Bar, Eatery]",Ayinger Privatbrauerei,http://ayinger-bier.de,Münchener Straße 21,85653,08095-90650,https://maps.google.com/maps?oi=map&q=M%C3%BCn...,None
1,United States,Pennsylvania,Hershey,"[Brewery, Bar, Eatery, Beer-to-go]",Tröegs Brewing Company,http://troegs.com,200 East,17033,(717) 534-1297,https://maps.google.com/maps?oi=map&q=200+East...,Sunday-Wednesday 11am-9pmThursday - sat 11am-1...
2,Germany,None,München,[Brewery],Spaten-Franziskaner-Bräu,http://franziskaner-weissbier.de,Mars Strasse 46-48,80335,(089) 51 221,https://maps.google.com/maps?oi=map&q=Mars+Str...,OWNED BY ANHEUSER-BUSCH INBEVSee also: http://...
3,Germany,None,München,[Brewery],Paulaner Brauerei,http://paulaner.de,Hochstraße 75,81541,089 / 4 80 051,https://maps.google.com/maps?oi=map&q=Hochstra...,None
4,Germany,None,Freising,"[Brewery, Bar, Eatery, Beer-to-go]",Bayerische Staatsbrauerei Weihenstephan,http://weihenstephaner.de,Alte Akademie 2,85354,+49 8161 5360,https://maps.google.com/maps?oi=map&q=Alte+Aka...,None


#### Get a foreign key for country_DIM and add it to the  brewery_df as a new column

In [486]:
# Define the country_DIM select query.
country_DIM_select_query = "SELECT * FROM [beer_db].[dbo].[country_DIM]"

# Execute the query.
country_DIM_select = cursor.execute(country_DIM_select_query).fetchall()

#Add FK_country_id column into brewery_df
brewery_df['FK_country_id'] = None
for row in country_DIM_select:
    if row[2]:
        second_condition = (brewery_df['brewery_state'] == row[2])
    else:
        second_condition = ~(brewery_df['brewery_state'] == row[2])
    country_id_mask = (brewery_df['brewery_country'] == row[1]) & second_condition
    brewery_df['FK_country_id'].loc[country_id_mask] = row[0]

#### Get a foreign key for city_DIM and add it to the  brewery_df as a new column

In [503]:
# Define the city_DIM select query.
city_DIM_select_query = "SELECT * FROM [beer_db].[dbo].[city_DIM]"

# Execute the query.
city_DIM_select = cursor.execute(city_DIM_select_query).fetchall()

#Add FK_city_id column into brewery_df
brewery_df['FK_city_id'] = None
for row in city_DIM_select:
    city_id_mask = brewery_df['brewery_city'] == row[1]
    brewery_df['FK_city_id'].loc[city_id_mask] = row[0]

#### Get a foreign key for brewery_type_DIM and add it to the  brewery_df as a new column

In [505]:
# Define the city_DIM select query.
brewery_type_DIM_select_query = "SELECT * FROM [beer_db].[dbo].[brewery_type_DIM]"

# Execute the query.
city_DIM_select = cursor.execute(brewery_type_DIM_select_query).fetchall()

#Add FK_city_id column into brewery_df
# brewery_df['FK_city_id'] = None
# for row in city_DIM_select:
#     city_id_mask = brewery_df['brewery_city'] == row[1]
#     brewery_df['FK_city_id'].loc[city_id_mask] = row[0]

[(1, True, True, True, True, True),
 (2, False, True, True, True, True),
 (3, True, False, True, True, True),
 (4, True, True, False, True, True),
 (5, True, True, True, False, True),
 (6, True, True, True, True, False),
 (7, False, False, True, True, True),
 (8, False, True, False, True, True),
 (9, False, True, True, False, True),
 (10, False, True, True, True, False),
 (11, True, False, False, True, True),
 (12, True, False, True, False, True),
 (13, True, False, True, True, False),
 (14, True, True, False, False, True),
 (15, True, True, False, True, False),
 (16, True, True, True, False, False),
 (17, False, False, False, True, True),
 (18, False, False, True, False, True),
 (19, False, False, True, True, False),
 (20, False, True, False, False, True),
 (21, False, True, False, True, False),
 (22, False, True, True, False, False),
 (23, True, False, False, False, True),
 (24, True, False, False, True, False),
 (25, True, False, True, False, False),
 (26, True, True, False, False, 

In [504]:
brewery_df

,brewery_country,brewery_state,brewery_city,brewery_type,brewery_name,brewery_website,brewery_adress,brewery_postal_code,brewery_telephone,brewery_map,brewery_notes,FK_country_id,FK_city_id
0,Germany,None,Aying,"[Brewery, Bar, Eatery]",Ayinger Privatbrauerei,http://ayinger-bier.de,Münchener Straße 21,85653,08095-90650,https://maps.google.com/maps?oi=map&q=M%C3%BCn...,None,60,254
1,United States,Pennsylvania,Hershey,"[Brewery, Bar, Eatery, Beer-to-go]",Tröegs Brewing Company,http://troegs.com,200 East,17033,(717) 534-1297,https://maps.google.com/maps?oi=map&q=200+East...,Sunday-Wednesday 11am-9pmThursday - sat 11am-1...,201,2199
2,Germany,None,München,[Brewery],Spaten-Franziskaner-Bräu,http://franziskaner-weissbier.de,Mars Strasse 46-48,80335,(089) 51 221,https://maps.google.com/maps?oi=map&q=Mars+Str...,OWNED BY ANHEUSER-BUSCH INBEVSee also: http://...,60,3515
3,Germany,None,München,[Brewery],Paulaner Brauerei,http://paulaner.de,Hochstraße 75,81541,089 / 4 80 051,https://maps.google.com/maps?oi=map&q=Hochstra...,None,60,3515
4,Germany,None,Freising,"[Brewery, Bar, Eatery, Beer-to-go]",Bayerische Staatsbrauerei Weihenstephan,http://weihenstephaner.de,Alte Akademie 2,85354,+49 8161 5360,https://maps.google.com/maps?oi=map&q=Alte+Aka...,None,60,1774
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12505,United Kingdom,England,Sudbury,"[Brewery, Beer-to-go]",Little Earth Project,http://littleearthproject.com,Behind White Horse PubEdwardstone,CO10 5PX,+44 01787211118,https://maps.google.com/maps?oi=map&q=Behind+W...,None,146,5085
12506,Belgium,None,Brugge,[Brewery],Pellicle Vergistingen,None,Sint-Arnolduslaan 72,None,None,https://maps.google.com/maps?oi=map&q=Sint-Arn...,None,15,669
12507,Italy,None,Villaga (Vi),[Brewery],Siemàn,http://sieman.it,"Via Crocenera, 13",36021,+39 340 222 4049,https://maps.google.com/maps?oi=map&q=Via+Croc...,None,78,5536
12508,United States,New York,New Paltz,"[Brewery, Bar, Eatery]",Bacchus & The Brewery At Bacchus,http://bacchusnewpaltz.com,4 South Chestnut St,12561,(845) 255-8636,https://maps.google.com/maps?oi=map&q=4+South+...,Brewery website: http://thebreweryatbacchus.com/,201,3610


In [ ]:
# beer_name
# name of style
# name_of_substyle
# rank_in_style
# score	rank
# abv [%]	
# avg_score
# deviation_score [%]	
# ratings	active	
# date_added	
# who_wants	
# who_have
# brewery_name